출처
- https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510
- https://dl.acm.org/doi/fullHtml/10.1145/3453476
- https://github.com/zszyellow/WER-in-python/blob/master/wer.py
- https://mingchin.tistory.com/240

OCR Metric
- Character Error Rate (CER)
    - D : 음성 인식된 텍스트에 잘못 삭제된 음절 수
    - S : 음성 인식된 텍스트에 잘못 대체된 음절 수
    - I : 음성 인식된 텍스트에 잘못 추가된 음절 수
    - N : 정답 텍스트의 음절 수
    > 음절 에러 비율(CER) = (S+D+I)/N
- Word Error Rate (WER)
    - D : 음성 인식된 텍스트에 잘못 삭제된 단어 수
    - S : 음성 인식된 텍스트에 잘못 대체된 단어 수
    - I : 음성 인식된 텍스트에 잘못 추가된 단어 수
    - N : 정답 텍스트의 단어 수
    > 단어 에러 비율(WER) = (S+D+I)/N      

논문 내용 중 일부
- Regarding NLP, several applications, i.e., named entity recognition (NER), part-of-speech (POS) tagging, text summarization, sentence boundary detection, topic modeling, sentiment analysis, text classification, named entity linking, and so on, are badly affected by OCR errors. Performance of NER tools, which locate proper names and categorise them into the set of predefined classes (i.e., person, location, organization), considerably degrades along with the increase in error rate (ER) of OCR output [59, 104, 158]. When the word error rate (WER) of the text increases from 0% to 2.7%, the F-score of the NER tool decreases around 3% [104]. With higher ER, the performance of NER drops much faster, for instance, from 90% to 60% when the WER of the text rises from 1% to 7% or when its character error rate (CER) increases from 8% to 20% [59].

    - NLP 관련 application들은 대부분 OCR 오류의 영향을 많이 받음
    - error rate of OCR output가 커질수록 성능 저하가 이루어짐.
        - word error rate(WER)이 증가하면, NER task의 경우 f1 score가 감소하게 됨.
        - character error rate(CER)도 마찬가지.


In [10]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=172276 sha256=7f85d5069106043dc83b431d2ae2f42d3fe80fb59f554ef9f77baf3749db4514
  Stored in directory: /opt/ml/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein


In [11]:
import sys
import numpy
import Levenshtein as Lev

In [3]:
def editDistance(r, h):
    """
    문자열 간의 유사도를 알아내는 edit distance 알고리즘을 적용하여 계산합니다.

    Attributes:
        r -> reference 문장을 split으로 분할하여 만든 단어 리스트
        h -> hypothesis 문장을 split으로 분할하여 만든 단어 리스트
    """
    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8).reshape(
        (len(r) + 1, len(h) + 1)
    )
    for i in range(len(r) + 1):
        d[i][0] = i
    for j in range(len(h) + 1):
        d[0][j] = j
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitute = d[i - 1][j - 1] + 1
                insert = d[i][j - 1] + 1
                delete = d[i - 1][j] + 1
                d[i][j] = min(substitute, insert, delete)
    return d

In [4]:
def getStepList(r, h, d):
    """
    동적 프로그래밍 프로세스에서 Step list를 가져옵니다.

    Attributes:
        r -> reference 문장을 split으로 분할하여 만든 단어 리스트
        h -> hypothesis 문장을 split으로 분할하여 만든 단어 리스트
        d -> h와 r의 edit distance를 계산할 때 만들어진 matrix
    """
    x = len(r)
    y = len(h)
    list = []
    while True:
        if x == 0 and y == 0:
            break
        elif x >= 1 and y >= 1 and d[x][y] == d[x - 1][y - 1] and r[x - 1] == h[y - 1]:
            list.append("e")
            x = x - 1
            y = y - 1
        elif y >= 1 and d[x][y] == d[x][y - 1] + 1:
            list.append("i")
            x = x
            y = y - 1
        elif x >= 1 and y >= 1 and d[x][y] == d[x - 1][y - 1] + 1:
            list.append("s")
            x = x - 1
            y = y - 1
        else:
            list.append("d")
            x = x - 1
            y = y
    return list[::-1]

In [5]:
def alignedPrint(list, r, h, result):
    """
    reference와 hypothesis 문장을 일렬로 비교한 결과를 출력합니다.

    Attributes:
        list   -> Step list
        r      -> reference 문장을 split으로 분할하여 만든 단어 리스트
        h      -> hypothesis 문장을 split으로 분할하여 만든 단어 리스트
        result -> edit distnace를 기반으로 계산된 rate
    """
    print("REF:", end=" ")
    for i in range(len(list)):
        if list[i] == "i":
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print(" " * (len(h[index])), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) < len(h[index2]):
                print(r[index1] + " " * (len(h[index2]) - len(r[index1])), end=" ")
            else:
                print(r[index1], end=" "),
        else:
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(r[index], end=" "),

    print("\nHYP:", end=" ")
    for i in range(len(list)):
        if list[i] == "d":
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(" " * (len(r[index])), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                print(h[index2] + " " * (len(r[index1]) - len(h[index2])), end=" ")
            else:
                print(h[index2], end=" ")
        else:
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print(h[index], end=" ")

    print("\nEVA:", end=" ")
    for i in range(len(list)):
        if list[i] == "d":
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print("D" + " " * (len(r[index]) - 1), end=" ")
        elif list[i] == "i":
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print("I" + " " * (len(h[index]) - 1), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                print("S" + " " * (len(r[index1]) - 1), end=" ")
            else:
                print("S" + " " * (len(h[index2]) - 1), end=" ")
        else:
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(" " * (len(r[index])), end=" ")
    print("\nWER: " + result)

In [6]:
def wer(r, h):
    """
    Word Error Rate(WER)을 계산합니다.

    Example :
    >>> wer("what is it".split(), "what is".split())
    >>> REF: what is it
        HYP: what is
        EVA:         D
        WER: 33.33%
    """
    # build the matrix
    d = editDistance(r, h)

    # find out the manipulation steps
    list = getStepList(r, h, d)

    # print the result in aligned way
    result = float(d[len(r)][len(h)]) / len(r) * 100
    result = str("%.2f" % result) + "%"
    alignedPrint(list, r, h, result)

In [7]:
wer("what is it".split(), "what is".split())

REF: what is it 
HYP: what is    
EVA:         D  
WER: 33.33%


In [ ]:
# def wer(ref, hyp ,debug=False):
#     r = ref.split()
#     h = hyp.split()
#     #costs will holds the costs, like in the Levenshtein distance algorithm
#     costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
#     # backtrace will hold the operations we've done.
#     # so we could later backtrace, like the WER algorithm requires us to.
#     backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

#     OP_OK = 0
#     OP_SUB = 1
#     OP_INS = 2
#     OP_DEL = 3

#     DEL_PENALTY=1 # Tact
#     INS_PENALTY=1 # Tact
#     SUB_PENALTY=1 # Tact
#     # First column represents the case where we achieve zero
#     # hypothesis words by deleting all reference words.
#     for i in range(1, len(r)+1):
#         costs[i][0] = DEL_PENALTY*i
#         backtrace[i][0] = OP_DEL

#     # First row represents the case where we achieve the hypothesis
#     # by inserting all hypothesis words into a zero-length reference.
#     for j in range(1, len(h) + 1):
#         costs[0][j] = INS_PENALTY * j
#         backtrace[0][j] = OP_INS

#     # computation
#     for i in range(1, len(r)+1):
#         for j in range(1, len(h)+1):
#             if r[i-1] == h[j-1]:
#                 costs[i][j] = costs[i-1][j-1]
#                 backtrace[i][j] = OP_OK
#             else:
#                 substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
#                 insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
#                 deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

#                 costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
#                 if costs[i][j] == substitutionCost:
#                     backtrace[i][j] = OP_SUB
#                 elif costs[i][j] == insertionCost:
#                     backtrace[i][j] = OP_INS
#                 else:
#                     backtrace[i][j] = OP_DEL

#     # back trace though the best route:
#     i = len(r)
#     j = len(h)
#     numSub = 0
#     numDel = 0
#     numIns = 0
#     numCor = 0
#     if debug:
#         print("OP\tREF\tHYP")
#         lines = []
#     while i > 0 or j > 0:
#         if backtrace[i][j] == OP_OK:
#             numCor += 1
#             i-=1
#             j-=1
#             if debug:
#                 lines.append("OK\t" + r[i]+"\t"+h[j])
#         elif backtrace[i][j] == OP_SUB:
#             numSub +=1
#             i-=1
#             j-=1
#             if debug:
#                 lines.append("SUB\t" + r[i]+"\t"+h[j])
#         elif backtrace[i][j] == OP_INS:
#             numIns += 1
#             j-=1
#             if debug:
#                 lines.append("INS\t" + "****" + "\t" + h[j])
#         elif backtrace[i][j] == OP_DEL:
#             numDel += 1
#             i-=1
#             if debug:
#                 lines.append("DEL\t" + r[i]+"\t"+"****")
#     if debug:
#         lines = reversed(lines)
#         for line in lines:
#             print(line)
#         print("Ncor " + str(numCor))
#         print("Nsub " + str(numSub))
#         print("Ndel " + str(numDel))
#         print("Nins " + str(numIns))
#     return numCor, numSub, numDel, numIns, (numSub + numDel + numIns) / (float) (len(r))

In [19]:
def cer(ref, hyp):
    # ref = ref.replace(' ', '')
    # hyp = hyp.replace(' ', '')
    dist = Lev.distance(hyp, ref)
    length = len(ref)
    return dist, length, dist/length

In [15]:
cer("what is it", "what is")

(2, 8, 0.25)

In [16]:
wer("my name is kenneth".split(), "my name is kenneth".split())

REF: my  name is kenneth 
HYP: myy nime iz kenneth 
EVA: S   S    S          
WER: 75.00%


In [20]:
cer("my name is kenneth", "myy nime iz kenneth")

(3, 18, 0.16666666666666666)